In [1]:
from src.content.produce import produce_content

themes = [ 'summer' , 'winter' ]
profile_name= "@the.wisdom.drops"
txt_aspect_ratio='WIDE'
n = 3 
produce_content(themes, posts_per_theme=n, profile_name=profile_name, txt_aspect_ratio=txt_aspect_ratio)

C:\Users\muril\.virtualenvs\insta-post-generator-N5ODM5nx\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\muril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[{'id': 'summer_0',
  'theme': 'summer',
  'filepath': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_0.PNG'),
  'filepath_txt': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_0.txt')},
 {'id': 'summer_1',
  'theme': 'summer',
  'filepath': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_1.PNG'),
  'filepath_txt': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_1.txt')},
 {'id': 'summer_2',
  'theme': 'summer',
  'filepath': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_2.PNG'),
  'filepath_txt': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/summer_2.txt')},
 {'id': 'winter_0',
  'theme': 'winter',
  'filepath': WindowsPath('c:/users/muril/documents/github/insta-post-generator/data/processed/winter_0.PNG'),
  'filepath_txt': WindowsPath('c:/users/muri